In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import glob
import openpyxl

In [2]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'


In [3]:
df = pd.read_csv(raw_dir+'aimdb.tsv', sep='\t', names=['SMILES','CASRN','MW','Other','AIM Fragments'])
df['AIM_list'] = df['AIM Fragments'].apply(lambda x: x.split(','))
df = df[['SMILES','AIM_list']].drop_duplicates(subset=['SMILES'])

In [4]:
db_list = []
for row in df['AIM_list']:
    for i in row:
        db_list.append(i)
        
db_uniques = len(set(db_list))

print(f"Number of unique fragments found in AIM Database: {db_uniques}")

Number of unique fragments found in AIM Database: 767


In [5]:
df_appendix = pd.read_excel(raw_dir+'AppendixA_Jun_09_2022.xlsx')

In [6]:
print(f"Number of fragment IDS found in AIM Appendix: {len(df_appendix)}")

Number of fragment IDS found in AIM Appendix: 831


In [7]:
app_names_list = list(set(df_appendix['Fragment'].values.tolist()))
app_names_list  = [str(x) for x in app_names_list ]
app_names_list = [x for x in app_names_list  if x != 'nan']

print(f"Number of UNIQUE Fragments found in AIM Appendix: {len(app_names_list)}")

Number of UNIQUE Fragments found in AIM Appendix: 782


In [8]:
df_csrml_fp = pd.read_csv(raw_dir+'AIM_V1_newest_names.tsv',sep='\t')
AIM_FP = df_csrml_fp.set_index('M_NAME')

In [9]:
n_subgraphs = df_csrml_fp.shape[1]
print(f"Number of SUBGRAPHS found in AIM CSRML: {n_subgraphs}")

Number of SUBGRAPHS found in AIM CSRML: 903


In [10]:
from collections import Counter
import re
print(AIM_FP.shape)

col_names = []
for i in AIM_FP.columns:
    col_names.append(i.split('.')[0])    

# Could add a condition to check that split[1] is a number, if not then check 

mylist = Counter(col_names)
duplicate_list= {key: value for key, value in mylist.items() if value > 1}

len_dict = len(duplicate_list)
sum_dict = sum(duplicate_list.values())
print(f'{sum_dict-len_dict} columns to be removed')

cols = []
for i in AIM_FP.columns:
    if i.split('.')[0] in duplicate_list.keys():
        cols.append(i)
        
df_dupes =AIM_FP[cols].copy()
for i,col in enumerate(df_dupes):
    col_1 = df_dupes.iloc[:,i].name.split('.')[0]
    # Match columns which are duplicates (e.g. Mercury [Measured] and Mercury [Measured].1 ) by name
    for j, col2 in enumerate(df_dupes):
        col_2 = df_dupes.iloc[:,j].name.split('.')[0]
            # If match, then column values will be updated to maximum value across the two columns for each row
        if col_1 == col_2:
            df_dupes[col] = df_dupes[[col,col2]].max(axis=1)
    
       
n_before = AIM_FP.shape[1]
for col in df_dupes:
    # Remove duplicate columns which are not the first (indicated by ending in  .[1-9])
    if re.search('\.[1-9]$',col) is not None:
        AIM_FP = AIM_FP.drop(columns=[col])
    else:
        AIM_FP[col] = df_dupes[col]
    
    

n_after = AIM_FP.shape[1]

print(f'{n_before-n_after} columns removed')


AIM_FP = AIM_FP.dropna(axis=1)
print(AIM_FP.shape)
print("Unique(?) Fragments in CSRML:", AIM_FP.shape[1])

(80, 902)
118 columns to be removed
118 columns removed
(80, 784)
Unique(?) Fragments in CSRML: 784


In [11]:
CSRML_unique= AIM_FP.columns.to_list()

In [12]:
df_csrml_fp2 = pd.read_csv(raw_dir+'AIM_V1_newest_names.tsv',sep='\t')
AIM_FP_full = df_csrml_fp2.set_index('M_NAME')
CSRML_names = AIM_FP_full.columns.tolist()

In [13]:
df_csrml_fp_id = pd.read_csv(raw_dir+'AIM_V1_newest_IDS.tsv',sep='\t').set_index('M_NAME')

In [14]:
CSRML_IDS = df_csrml_fp_id.columns.tolist()

In [15]:
CSRML_aim_dict = dict(zip(CSRML_IDS,CSRML_names))


In [16]:
unique_CSRML_ids = []
for i in CSRML_unique:
    for key,value in CSRML_aim_dict.items():
        if i == value:
            unique_CSRML_ids.append(key)

In [17]:
df_appendix = pd.read_excel(raw_dir+'AppendixA_Jun_09_2022.xlsx')

In [18]:
Appendix_Fragment_names = df_appendix['Fragment'].tolist()
Appendix_Fragment_IDS  = df_appendix['Number'].tolist()
Appendix_aim_dict = dict(zip(Appendix_Fragment_IDS,Appendix_Fragment_names))


In [19]:
app_names_list = list(set(df_appendix['Fragment'].values.tolist()))
app_names_list  = [str(x) for x in app_names_list ]
Appendix_unique = [x for x in app_names_list  if x != 'nan']


In [20]:
unique_Appendix_ids = []
for i in Appendix_unique:
    for key,value in Appendix_aim_dict.items():
        if i == value:
            unique_Appendix_ids.append(key)
            break

In [21]:
len(unique_Appendix_ids)
unique_Appendix_ids = [str(item) for item in unique_Appendix_ids]

In [22]:
A = [item for item in unique_CSRML_ids if item not in unique_Appendix_ids] 
B =  [item for item in unique_Appendix_ids if item not in unique_CSRML_ids] 

In [23]:
A


['93E', '97E', '98E', '99E', '96C_a', '73X']

In [24]:
B

['425C', '359C', '96C', '360C']

73X, 93E are the exclusive to A

- 97E = 425C
- 98E = 359C 
- 99E = 360C
- 96C_a = 96C

In [25]:
selection = ['73X']
filter_ = df['AIM_list'].apply(lambda x: any(item for item in selection if item in x))

len(df[filter_])

19

In [26]:
selection = ['93E']
filter_ = df['AIM_list'].apply(lambda x: any(item for item in selection if item in x))

len(df[filter_])

1